<a href="https://colab.research.google.com/github/elliottbonal/Z3metaphysicstests/blob/main/SeventhTheoremZ3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install z3-solver
from z3 import *

# Setting up the theorem solver
s = Solver()

# Defining the domain of all valid types (T143)
T143 = DeclareSort('T143')
x = Const('x', T143)

# Set-up for a reduced model of the Zmatrix being 5x5
N = 5
M = 5
Index = IntSort()

# Define predicates, Zmatrix and the zero_matrix
P = Function('P', Index, Index, T143, RealSort())
Z = Function('Z', T143, ArraySort(Index, ArraySort(Index, RealSort())))
zero_matrix = K(Index, K(Index, RealVal(0)))

# Defining row length and valid entries in the matrix
RowLength = Function('RowLength', Index, Index)
Valid = Function('Valid', Index, Index, BoolSort())

# Defining entries as valid if there are of row 1 or higher, and if column index j lies within the valid range of row i (not every row might have the same number of columns)
i, j = Ints('i j')
s.add(ForAll([i, j], Valid(i, j) == And(i >= 1, j >= 1, j <= RowLength(i))))

# Build matrix Z(x): Start with a Matrix full of zeros, and fill in each entry with predicate functions
Zx = K(Index, K(Index, RealVal(0)))
for i_val in range(1, N + 1):
    row = K(Index, RealVal(0))
    for j_val in range(1, M + 1):
        s.add(Valid(i_val, j_val) == (j_val <= RowLength(i_val)))
        row = Store(row, j_val, If(j_val <= RowLength(i_val), P(i_val, j_val, x), RealVal(0)))
    Zx = Store(Zx, i_val, row)
s.add(Z(x) == Zx)

# P_1.1 is the only predicate in the first row
for j_val in range(1, M + 1):
    if j_val != 1:
        s.add(P(1, j_val, x) == 0)
s.add(RowLength(1) == 1)

# Encoding that if a given row n is full of 0s then row n+1 is full of 0s as well
for i_val in range(2, N + 1):
    for j_val in range(1, M + 1):
        previous_zero = And([Implies(k <= RowLength(i_val - 1), P(i_val - 1, k, x) == 0)
                             for k in range(1, M + 1)])
        s.add(Implies(And(j_val <= RowLength(i_val), previous_zero),
                      P(i_val, j_val, x) == 0))

# Trying to falsify the claim that P_11 = 0 implies the Z matrix collapses (all entries become 0)
s.push()
s.add(Not(Implies(P(1, 1, x) == 0, Z(x) == zero_matrix)))

result = s.check()
if result == unsat:
    print("No counterexample possible, the theorem holds.")
else:
    print("The theorem fails, counterexample found.")
    print("Model showing inconsistency:")
    print(s.model())



No counterexample possible, the theorem holds.
